In [ ]:
%pip install python-Levenshtein

In [ ]:
import pickle
import tensorflow as tf
import numpy as np
import cv2
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Reshape, Dense, Bidirectional, Dropout, LSTM
from tensorflow.keras import Model
from tensorflow.keras.backend import ctc_decode
import Levenshtein as lev
from collections import Counter

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

with open('./drive/MyDrive/imgtensor_sh.pkl','rb') as arrbee:
    img_tensor_sh = pickle.load(arrbee)
    
with open('./labeltensor_shpad.pkl','rb') as ry:
    lab_tensor_sh = pickle.load(ry)
    



    
charlist = 'ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz .,-+1234567890'


def CTCLoss(y_true, y_pred):
    # Compute the training-time loss value
    batch_len = tf.cast(tf.shape(y_true)[0], dtype="int64")
    input_length = tf.cast(tf.shape(y_pred)[1], dtype="int64")
    label_length = tf.cast(tf.shape(y_true)[1], dtype="int64")

    input_length = input_length * tf.ones(shape=(batch_len, 1), dtype="int64")
    label_length = label_length * tf.ones(shape=(batch_len, 1), dtype="int64")

    loss = tf.keras.backend.ctc_batch_cost(y_true, y_pred, input_length, label_length)
    return loss


def decode_preds(pred_model,start,end):
    #preds = pred_model.predict(img_tensor_trsh[start:end])
    preds = pred_model
    len_tensor = tf.convert_to_tensor([len([l for l in lab if l>0]) for lab in lab_tensor_sh[start:end]])
    pred_texts = ctc_decode(preds,len_tensor,greedy=True)
    guess_list =  [''.join([charlist[p-1] for p in pred.numpy() if p > -1]) for pred in pred_texts[0][0]]
    prob_list = pred_texts[1].numpy()
    return guess_list,prob_list

def post_distance(pred_model,start,end):
    preds, _ = decode_preds(pred_model,start,end)
    trues = [''.join([charlist[p-1] for p in lab.numpy() if p>0]) for lab in lab_tensor_sh[start:end]]
    return np.mean([lev.distance(trues[i],preds[i]) for i in range(end-start)])

In [ ]:

!unzip fm2.zip -d ./firstmodel2/    
model2 = tf.keras.models.load_model('./firstmodel2/first_model_200', custom_objects={'CTCLoss':CTCLoss})

In [ ]:

hist2 = model2.fit(img_tensor_sh,lab_tensor_sh,validation_split=0.2, epochs=eps,callbacks=[early_stopping])

In [ ]:
pred_model = model2.predict(img_tensor_sh)
guesses,preds = decode_preds(pred_model,0,1407)

trues = trues = [''.join([charlist[p-1] for p in  lab.numpy() if p>0]) for lab in lab_tensor_sh[:1407]]

In [ ]:
post_distance(pred_model,0,1407)